In [1]:

import os
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.schema import Document
from langchain_openai import ChatOpenAI
import tiktoken


In [2]:

# Cargamos las variables de entorno
load_dotenv(find_dotenv(), override=True)


True

In [3]:

def cargar_documento(archivo):
    #Cargamos un documento PDF o DOCX y lo devolvemos en un formato procesable
    nombre, extension = os.path.splitext(archivo)
    if extension == '.pdf':
        print(f'Cargando {archivo}...')
        loader = PyPDFLoader(archivo)
    elif extension == '.docx':
        print(f'Cargando {archivo}...')
        loader = Docx2txtLoader(archivo)
    else:
        raise ValueError('El formato del documento no es soportado.')
    return loader.load()


In [4]:

def desde_wikipedia(busqueda, lang='es', load_max_docs=2):
    # Realizamos una búsqueda en Wikipedia y retornamos los resultados.
    loader = WikipediaLoader(query=busqueda, lang=lang, load_max_docs=load_max_docs)
    return loader.load()


In [5]:

def fragmentar(data, chunk_size=150, chunk_overlap=20):
    # Dividimos el texto en fragmentos más pequeños para el procesamiento
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_documents(data)


In [6]:

def costo_embedding(texts):
    # Calculamos el costo de embeddings en función del número de tokens.
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum(len(enc.encode(page.page_content)) for page in texts)
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0001:.5f}')


In [7]:

def creando_vectores(fragmentos, index_name):
    import time
    from pinecone import Pinecone, ServerlessSpec
    from langchain.schema import Document
    from langchain.embeddings.openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings()

    pc = Pinecone(api_key = os.environ.get('PINECONE_API_KEY'))
    
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

    from langchain.vectorstores import Pinecone
    if index_name not in existing_indexes:
        print(f'Creando el indice {index_name} y los embeddings ...', end = '')
        pc.create_index(name = index_name,
                        dimension = 1536, 
                        metric = 'cosine',
                        spec = ServerlessSpec(cloud = "aws", region = "us-east-1"),
                        )
        while not pc.describe_index(index_name).status["ready"]:
            time.sleep(1)
        vectores = Pinecone.from_documents(fragmentos, embeddings, index_name = index_name)
        print('Ok')
    else:
        print(f'El indice {index_name} ya existe. Cargando los embeddings ...', end = '')
        vectores = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')

    return vectores


In [8]:

def consultas(vectores, pregunta):
    # Realizamos consultas utilizando vectores y un modelo LLM.
    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
    retriever = vectores.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    return chain.run(pregunta)


In [9]:

def consulta_con_memoria(vectores, pregunta, memoria=[]):
    # Realizamos consultas con memoria de conversaciones previas.
    llm = ChatOpenAI(temperature=1)
    retriever = vectores.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    respuesta = crc({'question': pregunta, 'chat_history': memoria})
    memoria.append((pregunta, respuesta['answer']))
    return respuesta, memoria


In [10]:

def borrar_indices(index_name='todos'):
    from pinecone import Pinecone
    # Borramos índices de Pinecone según el nombre especificado.
    pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
    existing_indexes = [index["name"] for index in pinecone_client.list_indexes()]
    if index_name == 'todos':
        print('Borrando todos los índices ...')
        for index in existing_indexes:
            pinecone_client.delete_index(index)
        print('Listo!')
    elif index_name in existing_indexes:
        print(f'Borrando el índice {index_name} ...', end='')
        pinecone_client.delete_index(index_name)
        print('Listo')
    else:
        print(f'El índice {index_name} no existe.')


In [11]:

# Ejemplo de uso
documento = "Criptografía cuántica.pdf"
contenido = cargar_documento(documento)
fragmentos = fragmentar(contenido)
print(f"El número de fragmentos es de: {len(fragmentos)} fragmentos")
costo_embedding(fragmentos)
borrar_indices("todos")
index_name = 'criptografia-cuantica'
vectores = creando_vectores(fragmentos, index_name)

# Ciclo de preguntas
memoria = []
while True:
    pregunta = input("Realiza una pregunta, escribe 'salir' para terminar: \n")
    if pregunta.lower() == "salir":
        print("Adios!!!")
        break
    else:
        respuesta, memoria = consulta_con_memoria(vectores, pregunta, memoria)
        print(respuesta['answer'])


Cargando Criptografía cuántica.pdf...
El número de fragmentos es de: 144 fragmentos
Total Tokens: 3660
Embedding Cost in USD: 0.00037


c:\Users\josem\Documents\Python-Langchain\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Borrando todos los índices ...
Listo!


C:\Users\josem\AppData\Local\Temp\ipykernel_3056\3445364043.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


Creando el indice criptografia-cuantica y los embeddings ...Ok


C:\Users\josem\AppData\Local\Temp\ipykernel_3056\2109225436.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = crc({'question': pregunta, 'chat_history': memoria})


Una de las propiedades más importantes de la criptografía cuántica es la imposibilidad de conocer el valor exacto de dos propiedades complementarias al mismo tiempo.
La imposibilidad de conocer el valor exacto de dos propiedades complementarias al mismo tiempo en la criptografía cuántica se propuso inicialmente en 1984.
Adios!!!
